In [1]:
import numpy as np
import graphinglib as gl
import dill
import pyregion

from src.statistics.advanced_stats import increments
from src.hdu.maps.map import Map

In [2]:
def get_plottables(data: np.ndarray, number_of_bins: int, increment: float) -> [gl.Histogram, gl.Curve, gl.Text]:
    sorted_data = np.sort(np.concatenate((data, -data)))

    bin_width = 0.5
    # bin_width = 125
    bins = np.arange(np.nanmin(sorted_data), np.nanmax(sorted_data) + bin_width*0.99, bin_width)
    max_bin = np.nanmax(sorted_data + bin_width)//bin_width * bin_width
    bins = np.arange(-max_bin, max_bin, bin_width)
    hist_values, hist_edges = np.histogram(sorted_data, bins=bins)

    hist = gl.Histogram(
        sorted_data,
        number_of_bins=bins,
        # number_of_bins=number_of_bins,
        show_params=False
    )
    amplitude = np.max(hist.bin_heights)

    stddev = (hist_edges[np.argmin(np.abs(hist_values - np.max(hist_values)/2)) + 1] - bin_width/2) \
           / (np.sqrt(2*np.log(2)))
    # stddev = hist_edges[np.argmin(np.abs(hist_values - np.max(hist_values)/2)) + 1] / (np.sqrt(2*np.log(2)))
    # Half the bin_width could be added so the gaussian passes in the middle of the bins

    curve = gl.Curve.from_function(
        func=lambda x: amplitude * np.exp(-x**2/(2*stddev**2)),
        x_min=hist.bin_edges[0],
        x_max=hist.bin_edges[-1]
    )

    text = gl.Text(
        hist.bin_edges[0]*0.8,
        np.max(hist.bin_heights)*0.8,
        rf"$\Delta={increment}$"
    )

    return [hist, curve, text]

### All increments

In [3]:
turbulence = Map.load("summer_2023/gaussian_fitting/maps/computed_data_selective/turbulence.fits")

regions = [
    ("global_region", None, 40),
    ("diffuse_region", pyregion.open("summer_2023/gaussian_fitting/regions/region_1.reg"), 30),
    ("central_region", pyregion.open("summer_2023/gaussian_fitting/regions/region_2.reg"), 20),
    ("filament_region", pyregion.open("summer_2023/gaussian_fitting/regions/region_3.reg"), 10)
]

In [4]:
# with open("applications/sh158/saved_data/increments.gz", "rb") as f:
#     increments_data = dill.load(f)

all_figs = []

VAL = 0.5

for name, region, number_of_bins in regions:
    increments_data = increments(turbulence.get_masked_region(region).data)

    figs = []

    for increment in [2,5,10,20]:
        fig = gl.Figure(x_label="Increment", y_label="PDF", title=name)
        data = []
        for increment_i, values in increments_data.items():
            if increment + VAL > increment_i > increment - VAL:
                data.extend(values.tolist())
        fig.add_elements(*get_plottables(np.array(data), number_of_bins, increment))
        # fig.add_elements(*get_plottables(increments_data[increment], number_of_bins, increment))
        figs.append(fig)
        
    all_figs.extend(figs)

    multifig = gl.MultiFigure.from_grid(figs, (2,2), (14, 9))
    # multifig.show()
    # multifig.save(f"figures/sh158/advanced_stats/increments/{name}.pdf")

giga_multifig = gl.MultiFigure.from_grid(all_figs, (4,4), (20,11))
giga_multifig.save(f"figures/sh158/advanced_stats/increments/everything_new.pdf")

Looping started
Looping finished
Regroupment started
Regroupment finished
Looping started
Looping finished
Regroupment started
Regroupment finished
Looping started
Looping finished
Regroupment started
Regroupment finished
Looping started
Looping finished
Regroupment started
Regroupment finished


In [5]:
""" temperature = Map.load("summer_2023/gaussian_fitting/maps/external_maps/"
                      +"dens_it_sii_sans_fcorr_nii_plus_plusmin_pouss_seuil_errt_1000.fits")

alignment_map = (turbulence * 0).get_reprojection_on(temperature)

temperature += alignment_map

with open("applications/sh158/increments_temperature_2.gz", "wb") as f:
    dill.dump(increments(temperature.data), f)

all_figs = []

VAL = 0.5

for name, region, number_of_bins in regions:
    if not region:
        with open("applications/sh158/increments_temperature_2.gz", "rb") as f:
            increments_data = dill.load(f)
    else:
        increments_data = increments(temperature.get_masked_region(region).data)

    print("fig")
    figs = []

    for increment in [2,5,10,20]:
        fig = gl.Figure(x_label="Increment", y_label="PDF", title=name)
        data = []
        for increment_i, values in increments_data.items():
            if increment + VAL > increment_i > increment - VAL:
                data.extend(values.tolist())
        fig.add_elements(*get_plottables(np.array(data), number_of_bins, increment))
        # fig.add_elements(*get_plottables(increments_data[increment], number_of_bins, increment))
        figs.append(fig)
        
    all_figs.extend(figs)

    multifig = gl.MultiFigure.from_grid(figs, (2,2), (14, 9))
    # multifig.show()
    multifig.save(f"figures/sh158/advanced_stats/increments_temperature/{name}.pdf")

giga_multifig = gl.MultiFigure.from_grid(all_figs, (4,4), (20,11))
giga_multifig.save(f"figures/sh158/advanced_stats/increments_temperature/everything.pdf") """

' temperature = Map.load("summer_2023/gaussian_fitting/maps/external_maps/"\n                      +"dens_it_sii_sans_fcorr_nii_plus_plusmin_pouss_seuil_errt_1000.fits")\n\nalignment_map = (turbulence * 0).get_reprojection_on(temperature)\n\ntemperature += alignment_map\n\nwith open("applications/sh158/increments_temperature_2.gz", "wb") as f:\n    dill.dump(increments(temperature.data), f)\n\nall_figs = []\n\nVAL = 0.5\n\nfor name, region, number_of_bins in regions:\n    if not region:\n        with open("applications/sh158/increments_temperature_2.gz", "rb") as f:\n            increments_data = dill.load(f)\n    else:\n        increments_data = increments(temperature.get_masked_region(region).data)\n\n    print("fig")\n    figs = []\n\n    for increment in [2,5,10,20]:\n        fig = gl.Figure(x_label="Increment", y_label="PDF", title=name)\n        data = []\n        for increment_i, values in increments_data.items():\n            if increment + VAL > increment_i > increment - VAL:\